![rustronomy_dark_banner](https://github.com/smups/rustronomy/blob/main/logos/Rustronomy-fits_github_banner_dark.png?raw=true#gh-light-mode-only)
# Example 2 - plotting a FITS image
>_Note: this example assumes you have worked through example #1_

In [9]:
:dep rustronomy-fits = {git = "https://github.com/smups/rustronomy"}
:dep ndarray = {ndarray = 0.15, features = ["blas", "approx", "std"]}
:dep plotters = {git = "https://github.com/38/plotters", default_features = true, features = ["evcxr", "all_series"]}
:dep dirs = {dirs = 4}
:dep num-traits = {num-traits = 0.2}

In [10]:
use std::{
    fs,
    path,
    error::Error
};
use rustronomy_fits as rsf;
use ndarray as nd;
use dirs;
use plotters::prelude::*;
use num_traits::{Float, Num};

### 2.1 - Reading the FITS file
First, let's read the Image in the same manner we did last time:

In [11]:
//We'll be using one of the Hubble FITS files from the resources folder
let mut path = dirs::home_dir().unwrap();
path.push("Desktop/code/rustronomy/rustronomy/rustronomy-fits/resources/Hubble_NICMOS.fits");

//get the data from the file
let mut fits = rsf::Fits::open(&path)?;
let (header, data) = fits.remove_hdu(1).unwrap().to_parts();
let array = match data.unwrap() {
    rsf::Extension::Image(img) => img.as_owned_f32_array()?,
    _ => panic!()
};

If we want to turn this image into a grey-scale plot, we have to know the range
of the parameters inside the image first. For starters, let's convert the array
we found to a 2D array

_Comment: by default, rsf returns a dynamically sized ndarray (shape = IxDyn), which is
computationally expensive. Since we know the shape of the array (2D), we might
as well convert it to a statically sized 2D array_

In [12]:
let img = array.into_dimensionality::<nd::Ix2>()?;

### 2.2 - Creating a greyscale using plotters

Next we have to obtain the largest and smallest values in the array. Since `f64`
does not implement the `Ordered` trait, we can't use the built-in `max` and `min`
funcs. Luckily, rust is fast, so just looping over the iterators is fine too!

In [13]:
//Calculate max and min values
let mut max: f32 = f32::MIN;
for val in img.iter() { if val > &max {max = *val} }

let mut min: f32 = f32::MAX;
for val in img.iter() { if val < &min {min = *val} }
println!("max:{max}, min:{min}");

max:2730.8992, min:-1.0890907


Most of the time when viewing astronomical images it is advisable to map the raw
counts to a logarithmic scale, since our eyes don't perceive brightness differences
linearly, but logarithmically. Hence, we'll be grey-scaling our images as:
$$
    B(c)\in[0,255], \quad B(c)=255\left(\frac{\ln{c}-\ln{c_{min}}}{\ln{c_{max}}}\right)
    =255\left[\ln{\left(\frac{c}{c_{min}}\right)}\middle/\ln{c_{max}}\right]
$$
Ndarray provides us with an `indexed_iter()` method on the 2D array which we may
handily use to convert the 2D ndarray into pixels with a little function:

In [14]:
fn grey_scale(count: f32, min: f32, log_max: f32)
    -> Result<RGBColor, Box<dyn Error>>
{
    let col: u8 =
    (//This should be within the 0-255 range!
        255. * (count/min).abs().log10() / log_max
    ) as u8;

    Ok(RGBColor(col, col, col))
}

### 2.3 - Plotting the Image


In [15]:
//For each pixel in the image we want one pixel
let x_size = img.shape()[0];
let y_size = img.shape()[1];

{ //cursed extra curly braces
let root = BitMapBackend::new("ex2_image.png", (x_size as u32, y_size as u32)).into_drawing_area();
root.fill(&RED)?;

//create base chart with the size of the image
let mut chart = ChartBuilder::on(&root).build_cartesian_2d(0..x_size, 0..y_size)?;

//Remove the mesh from the chart
chart.configure_mesh()
    .disable_x_mesh()
    .disable_y_mesh()
    .draw()?;

let plotting_area = chart.plotting_area();

for ((x,y), count) in img.indexed_iter() {
    plotting_area.draw_pixel((x, y), &grey_scale(*count, min, max.log10())?)?
}
}

()

Output:

![](ex2_image.png)